In [31]:
!wget https://raw.githubusercontent.com/sisyphusinthegit/job-bot/main/config.json

import json

def load_config(path="config.json"):
    with open(path, "r") as f:
        return json.load(f)

config = load_config()
print(config["telegram"]["bot_token"])  # test

--2025-07-05 19:05:51--  https://raw.githubusercontent.com/sisyphusinthegit/job-bot/main/config.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 671 [text/plain]
Saving to: ‘config.json’

config.json         100%[===================>]     671  --.-KB/s    in 0s      

2025-07-05 19:05:51 (34.5 MB/s) - ‘config.json’ saved [671/671]

7616042675:AAGf10TxZoIBLTUoBu4SecHl-aMuVFU5JiI


In [ ]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

def load_config(path="config.json"):
    with open(path, "r") as f:
        return json.load(f)

def send_email(subject, body):
    config = load_config()
    email_conf = config["email"]

    msg = MIMEMultipart()
    msg["From"] = email_conf["sender_email"]
    msg["To"] = email_conf["receiver_email"]
    msg["Subject"] = subject
    msg.attach(MIMEText(body, "plain"))

    with smtplib.SMTP(email_conf["smtp_server"], email_conf["smtp_port"]) as server:
        server.starttls()
        server.login(email_conf["sender_email"], email_conf["sender_password"])
        server.send_message(msg)

    print("✅ Mail gönderildi.")

In [36]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

def load_config(path = 'config.json'):
  with open(path, 'r') as file:
    return json.load(file)

def send_email(subject,body):
  config = load_config()
  email_conf = config["email"]

  msg = MIMEMultipart()
  msg['From'] = email_conf['sender_email']
  msg['To'] = email_conf['receiver_email']
  msg['Subject'] = subject
  msg.attach(MIMEText(body,'plain'))

  with smtplib.SMTP(email_conf['smtp_server'],email_conf['smtp_port']) as server:
    server.starttls()
    server.login(email_conf['sender_email'],email_conf['sender_password'])
    server.send_message(msg)


  print("Email is sent successfully.")